In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch

Create raw data

In [2]:
from metal.utils import hard_to_soft

N = 1200
X = np.random.random((N,2)) * 2 - 1
Y = (X[:,0] > X[:,1] + 0.5).astype(int) + 1

X = torch.tensor(X, dtype=torch.float)
Y = torch.tensor(Y, dtype=torch.int16)

Separate into splits

In [3]:
X_train = X[:1000]
X_dev = X[1000:1100]
X_test = X[1100:]

Y_train = Y[:1000]
Y_dev = Y[1000:1100]
Y_test = Y[1100:]

In [4]:
from collections import Counter
print(Counter(Y_train.numpy()))
print(Counter(Y_dev.numpy()))
print(Counter(Y_test.numpy()))

Counter({1: 719, 2: 281})
Counter({1: 71, 2: 29})
Counter({1: 77, 2: 23})


Train and score

In [12]:
from metal.end_model import EndModel

em = EndModel()
em.train(X_train, Y_train, X_dev, Y_dev)

EndModel(
  (net): Sequential(
    (0): Linear(in_features=2, out_features=4, bias=True)
    (1): Linear(in_features=4, out_features=2, bias=True)
  )
  (criteria): SoftCrossEntropyLoss()
)
[E:1]	Train Loss: 0.006	Dev score: 0.690
[E:2]	Train Loss: 0.004	Dev score: 0.840
[E:3]	Train Loss: 0.003	Dev score: 0.780
[E:4]	Train Loss: 0.003	Dev score: 0.810
[E:5]	Train Loss: 0.003	Dev score: 0.860
[E:6]	Train Loss: 0.002	Dev score: 0.910
[E:7]	Train Loss: 0.002	Dev score: 0.920
[E:8]	Train Loss: 0.002	Dev score: 0.950
[E:9]	Train Loss: 0.002	Dev score: 0.970
[E:10]	Train Loss: 0.001	Dev score: 0.980


In [13]:
em.score(X_test, Y_test)

Accuracy: 0.960


0.96